In [9]:
from __future__ import print_function
from PIL import Image, ImageChops
import math
import numpy as np
import imageio as io
from skimage import img_as_float
from skimage.measure import compare_mse as mse
from skimage.util import crop
import SimpleITK as sitk
%matplotlib inline
from matplotlib import pyplot as plt
from ipywidgets import interact, fixed

#Having worked put the translations needed for a each tomogram to register the image 
#onto the binary mask (see part 2 of these notebooks) we now need to actually apply these to the volumes

point = (1.0, 1.0, 1.0)
dimension = 3 #i.e 3D

#these params where calculateed in the previous notebook from the registration process
param_0 = (0.0, 0.0, 0.0)
param_1 = (0.0, -1.0000499999999999, 0.0)
param_2 = (0.0, -1.13339, 0.0)
param_3 = (0.0, -1.0000499999999999, 0.0)
param_4 = (0.0, -1.0000499999999999, 0.0)
param_5 = (-1.0000499999999999, -1.0000499999999999, 0.0)
param_6 = (-1.0000499999999999, -1.0000499999999999, 0.0)
param_7 = (-1.0000499999999999, -1.0000499999999999, 0.0)
param_8 = (-1.0000499999999999, -1.0000499999999999, -0.8000399999999999)
param_9 = (-1.0000499999999999, -1.0000499999999999, -1.0000499999999999)
param_10 = (-1.0000499999999999, -1.0000499999999999, -1.0000499999999999)
param_11 = (0.0, 0.6666999999999998, -1.0000499999999999)
param_12 = (0.0, 2.0000999999999998, -1.0000499999999999)
param_13 = (-1.0000499999999999, 0.33335000000000004, 0.0)
param_14 = (-1.0000499999999999, 5.551115123125783e-17, 0.0)
param_15 = (-0.06667000000000001, 5.551115123125783e-17, 0.0)
param_16 = (-1.0000499999999999, 1.0000499999999999, -1.0000499999999999)
param_17 = (0.0, 1.0000499999999999, -1.0000499999999999)

#make lists
parameters = [param_0, param_1, param_2, param_3, param_4, param_5, param_6, param_7, param_8,
              param_9, param_10, param_11, param_12, param_13, param_14, param_15, param_16, param_17]
resampled_list = ['r_0', 'r_1', 'r_2', 'r_3', 'r_4', 'r_5', 'r_6', 'r_7', 'r_8', 'r_9', 'r_10', 'r_11', 'r_12',
                  'r_13', 'r_14', 'r_15', 'r_16', 'r_17']

#this function executes the resampling
def resample(image, transform):
    # Output image Origin, Spacing, Size, Direction are taken from the reference
    # image in this call to Resample
    reference_image = image
    interpolator = sitk.sitkNearestNeighbor
    default_value = 0
    return sitk.Resample(image, reference_image, transform,
                         interpolator, default_value)
    
for x in range(0, 18):
    image = sitk.ReadImage("E:/paper_2/Registration_Volumes/cropped_TiffSaver-tomo_C%d.tif"%(x), sitk.sitkFloat32)
    translation = sitk.TranslationTransform(dimension)
    translation.SetParameters(parameters[x])
    translation.SetOffset(parameters[x])
    resampled_list[x] = resample(image, translation)


In [11]:
print(parameters[17])


(0.0, 1.0000499999999999, -1.0000499999999999)


In [ ]:
#Now we will load in subsequent volumes and apply binary mask to them in a loop
#The mask was made in part 1 of these jupyter notebooks
thresh_deposit = 113
thresh_deposit_blur = 0.4
for x in range(2, 3):
    vol_xa = io.volread("E:/paper_2/Volumes_Cat/TiffSaver-tomo_C%d.tif" % (x))
    vol_xb = crop(vol_xa, ((0, 1510),(870, 5), (270, 255)), copy=True)
    vol_xc = gaussian(vol_xb, sigma=1.65, output=None, mode='nearest', cval=0, multichannel=False, preserve_range=True,
                       truncate=4.0)
    vol_xd = vol_xc * mask
    vol_xe = vol_xd > thresh_deposit
    vol_xf = gaussian(vol_xe, sigma=1.99, output=None, mode='nearest', cval=0, multichannel=False, preserve_range=True,
                       truncate=4.0)
    vol_xg = vol_xf > thresh_deposit_blur
    vol_x = 250*vol_xg + 100*segmented_yen_eroded_dilated_vol_0
    fig, axes = plt.subplots(nrows=6, ncols=1,figsize = (60, 18.4),
                       sharex=True, sharey=True)
axes[0].imshow(vol_xb[50], cmap = 'gray')
axes[1].imshow(vol_xd[50], cmap = 'gray')
axes[2].imshow(vol_xe[50], cmap = 'gray')
axes[3].imshow(vol_xf[50], cmap = 'gray')
axes[4].imshow(vol_xg[50], cmap = 'gray')
axes[5].imshow(vol_x[50], cmap = 'gray')